# Handling Missing Data

The difference between data found in many tutorials and data in the real world is that real-world data is rarely clean and homogeneous.
In particular, many interesting datasets will have some amount of data missing.
To make matters even more complicated, different data sources may indicate missing data in different ways.

In this section, we will discuss some general considerations for missing data, discuss how Pandas chooses to represent it, and demonstrate some built-in Pandas tools for handling missing data in Python.
Here and throughout the book, we'll refer to missing data in general as *null*, *NaN*, or *NA* values.


Manejo de datos faltantes





La diferencia entre los datos encontrados en muchos tutoriales y los datos del mundo real es que los datos del mundo real rara vez son limpios y homogéneos. En particular, muchos conjuntos de datos interesantes tendrán cierta cantidad de datos faltantes. Para complicar aún más las cosas, diferentes fuentes de datos pueden indicar datos faltantes de diferentes maneras.

En esta sección, discutiremos algunas consideraciones generales para los datos faltantes, discutiremos cómo Pandas elige representarlos y demostraremos algunas herramientas integradas de Pandas para manejar datos faltantes en Python. Aquí y a lo largo del libro, nos referiremos a los datos faltantes en general como valores nulos, NaN o NA.

## Trade-Offs in Missing Data Conventions

There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame.
Generally, they revolve around one of two strategies: using a *mask* that globally indicates missing values, or choosing a *sentinel value* that indicates a missing entry.

In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinel value could be some data-specific convention, such as indicating a missing integer value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number), a special value which is part of the IEEE floating-point specification.

None of these approaches is without trade-offs: use of a separate mask array requires allocation of an additional Boolean array, which adds overhead in both storage and computation. A sentinel value reduces the range of valid values that can be represented, and may require extra (often non-optimized) logic in CPU and GPU arithmetic. Common special values like NaN are not available for all data types.

As in most cases where no universally optimal choice exists, different languages and systems use different conventions.
For example, the R language uses reserved bit patterns within each data type as sentinel values indicating missing data, while the SciDB system uses an extra byte attached to every cell which indicates a NA state.


Compensaciones en convenios de datos faltantes
Hay una serie de esquemas que se han desarrollado para indicar la presencia de datos faltantes en una tabla o DataFrame. En general, giran en torno a una de dos estrategias: usar una máscara que indique globalmente valores perdidos o elegir un valor centinela que indique una entrada faltante.

En el enfoque de enmascaramiento, la máscara puede ser una matriz booleana completamente separada, o puede implicar la apropiación de un bit en la representación de datos para indicar localmente el estado nulo de un valor.

En el enfoque centinela, el valor centinela podría ser una convención específica de datos, como indicar un valor entero perdido con -9999 o algún patrón de bits raro, o podría ser una convención más global, como indicar un valor de punto flotante perdido con NaN (no es un número), un valor especial que forma parte de la especificación de punto flotante IEEE.

Ninguno de estos enfoques está exento de compensaciones: el uso de una matriz de máscara separada requiere la asignación de una matriz booleana adicional, que agrega una sobrecarga en el almacenamiento y la computación. Un valor centinela reduce el rango de valores válidos que se pueden representar y puede requerir una lógica adicional (a menudo no optimizada) en la aritmética de CPU y GPU. Los valores especiales comunes como NaN no están disponibles para todos los tipos de datos.

Como en la mayoría de los casos donde no existe una opción universalmente óptima, diferentes lenguajes y sistemas usan diferentes convenciones. Por ejemplo, el lenguaje R usa patrones de bits reservados dentro de cada tipo de datos como valores centinela que indican datos faltantes, mientras que el sistema SciDB usa un byte adicional adjunto a cada celda que indica un estado de NA.

## Missing Data in Pandas

The way in which Pandas handles missing values is constrained by its reliance on the NumPy package, which does not have a built-in notion of NA values for non-floating-point data types.

Pandas could have followed R's lead in specifying bit patterns for each individual data type to indicate nullness, but this approach turns out to be rather unwieldy.
While R contains four basic data types, NumPy supports *far* more than this: for example, while R has a single integer type, NumPy supports *fourteen* basic integer types once you account for available precisions, signedness, and endianness of the encoding.
Reserving a specific bit pattern in all available NumPy types would lead to an unwieldy amount of overhead in special-casing various operations for various types, likely even requiring a new fork of the NumPy package. Further, for the smaller data types (such as 8-bit integers), sacrificing a bit to use as a mask will significantly reduce the range of values it can represent.

NumPy does have support for masked arrays – that is, arrays that have a separate Boolean mask array attached for marking data as "good" or "bad."
Pandas could have derived from this, but the overhead in both storage, computation, and code maintenance makes that an unattractive choice.

With these constraints in mind, Pandas chose to use sentinels for missing data, and further chose to use two already-existing Python null values: the special floating-point ``NaN`` value, and the Python ``None`` object.
This choice has some side effects, as we will see, but in practice ends up being a good compromise in most cases of interest.


Datos faltantes en pandas
La forma en que Pandas maneja los valores perdidos está limitada por su dependencia del paquete NumPy, que no tiene una noción incorporada de los valores de NA para los tipos de datos de punto no flotante.

Los pandas podrían haber seguido el ejemplo de R al especificar patrones de bits para cada tipo de datos individual para indicar nulidad, pero este enfoque resulta bastante difícil de manejar. Si bien R contiene cuatro tipos de datos básicos, NumPy admite mucho más que esto: por ejemplo, mientras R tiene un solo tipo entero, NumPy admite catorce tipos enteros básicos una vez que tenga en cuenta las precisiones disponibles, la firma y la endianidad de la codificación. La reserva de un patrón de bits específico en todos los tipos de NumPy disponibles conduciría a una cantidad de gastos indirectos difícil de manejar en varias operaciones de revestimiento especial para varios tipos, probablemente incluso requiriendo una nueva bifurcación del paquete NumPy. Además, para los tipos de datos más pequeños (como los enteros de 8 bits), sacrificar un bit para usarlo como máscara reducirá significativamente el rango de valores que puede representar.

NumPy tiene soporte para matrices enmascaradas, es decir, matrices que tienen una matriz de máscara booleana separada adjunta para marcar datos como "buenos" o "malos". Los pandas podrían haberse derivado de esto, pero la sobrecarga en el almacenamiento, el cómputo y el mantenimiento del código hace que sea una opción poco atractiva.

Con estas restricciones en mente, Pandas eligió usar centinelas para los datos faltantes, y eligió usar dos valores nulos de Python ya existentes: el valor especial NaN de punto flotante y el objeto Python None. Esta elección tiene algunos efectos secundarios, como veremos, pero en la práctica termina siendo un buen compromiso en la mayoría de los casos de interés.

### ``None``: Pythonic missing data

The first sentinel value used by Pandas is ``None``, a Python singleton object that is often used for missing data in Python code.
Because it is a Python object, ``None`` cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type ``'object'`` (i.e., arrays of Python objects):

Ninguno: datos faltantes pitónicos
El primer valor centinela utilizado por Pandas es None, un objeto único de Python que a menudo se usa para datos faltantes en el código de Python. Debido a que es un objeto Python, None no se puede usar en ninguna matriz arbitraria NumPy / Pandas, sino solo en matrices con el tipo de datos 'objeto' (es decir, matrices de objetos Python):

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

This ``dtype=object`` means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects.
While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types:


Este dtype = object significa que la mejor representación de tipo común que NumPy podría inferir para el contenido de la matriz es que son objetos de Python. Si bien este tipo de matriz de objetos es útil para algunos fines, cualquier operación en los datos se realizará a nivel de Python, con mucha más sobrecarga que las operaciones típicamente rápidas que se ven para las matrices con tipos nativos:



In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
10 loops, best of 3: 78.2 ms per loop

dtype = int
100 loops, best of 3: 3.06 ms per loop



The use of Python objects in an array also means that if you perform aggregations like ``sum()`` or ``min()`` across an array with a ``None`` value, you will generally get an error:


El uso de objetos Python en una matriz también significa que si realiza agregaciones como sum () o min () en una matriz con un valor None, generalmente obtendrá un error:

In [4]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

This reflects the fact that addition between an integer and ``None`` is undefined.

Esto refleja el hecho de que la suma entre un entero y Ninguno no está definida

### ``NaN``: Missing numerical data

The other missing data representation, ``NaN`` (acronym for *Not a Number*), is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:


NaN: datos numéricos faltantes
La otra representación de datos faltantes, NaN (acrónimo de Not a Number), es diferente; Es un valor especial de punto flotante reconocido por todos los sistemas que utilizan la representación estándar de punto flotante IEEE:

In [5]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

Notice that NumPy chose a native floating-point type for this array: this means that unlike the object array from before, this array supports fast operations pushed into compiled code.
You should be aware that ``NaN`` is a bit like a data virus–it infects any other object it touches.
Regardless of the operation, the result of arithmetic with ``NaN`` will be another ``NaN``:

Tenga en cuenta que NumPy eligió un tipo de punto flotante nativo para esta matriz: esto significa que, a diferencia de la matriz de objetos de antes, esta matriz admite operaciones rápidas introducidas en código compilado. Debe tener en cuenta que NaN es un poco como un virus de datos: infecta cualquier otro objeto que toque. Independientemente de la operación, el resultado de la aritmética con NaN será otro NaN:

In [6]:
1 + np.nan

nan

In [7]:
0 *  np.nan

nan

Note that this means that aggregates over the values are well defined (i.e., they don't result in an error) but not always useful:

Tenga en cuenta que esto significa que los agregados sobre los valores están bien definidos (es decir, no producen un error) pero no siempre son útiles:

In [8]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy does provide some special aggregations that will ignore these missing values:


NumPy proporciona algunas agregaciones especiales que ignorarán estos valores faltantes:

In [9]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

Keep in mind that ``NaN`` is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types.


Tenga en cuenta que NaN es específicamente un valor de punto flotante; no hay un valor equivalente de NaN para enteros, cadenas u otros tipos.

### NaN and None in Pandas

``NaN`` and ``None`` both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

NaN y None en Pandas
NaN y None tienen su lugar, y Pandas está diseñado para manejarlos a ambos casi de manera intercambiable, convirtiendo entre ellos cuando sea apropiado:

In [10]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present.
For example, if we set a value in an integer array to ``np.nan``, it will automatically be upcast to a floating-point type to accommodate the NA:

Para los tipos que no tienen un valor centinela disponible, Pandas automáticamente convierte cuando los valores NA están presentes. Por ejemplo, si establecemos un valor en una matriz entera en np.nan, se convertirá automáticamente en un tipo de punto flotante para acomodar el NA:


In [11]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [12]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Notice that in addition to casting the integer array to floating point, Pandas automatically converts the ``None`` to a ``NaN`` value.
(Be aware that there is a proposal to add a native integer NA to Pandas in the future; as of this writing, it has not been included).

While this type of magic may feel a bit hackish compared to the more unified approach to NA values in domain-specific languages like R, the Pandas sentinel/casting approach works quite well in practice and in my experience only rarely causes issues.

The following table lists the upcasting conventions in Pandas when NA values are introduced:

Tenga en cuenta que, además de convertir la matriz de enteros en coma flotante, Pandas convierte automáticamente el Ninguno en un valor NaN. (Tenga en cuenta que hay una propuesta para agregar un NA nativo entero a Pandas en el futuro; a partir de este escrito, no se ha incluido).

Si bien este tipo de magia puede parecer un poco difícil en comparación con el enfoque más unificado de los valores de NA en lenguajes específicos de dominio como R, el enfoque de centinela / lanzamiento de Pandas funciona bastante bien en la práctica y, en mi experiencia, rara vez causa problemas.

La siguiente tabla enumera las convenciones de conversión en Pandas cuando se introducen los valores de NA


|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

Keep in mind that in Pandas, string data is always stored with an ``object`` dtype.

## Operating on Null Values

As we have seen, Pandas treats ``None`` and ``NaN`` as essentially interchangeable for indicating missing or null values.
To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures.
They are:

- ``isnull()``: Generate a boolean mask indicating missing values
- ``notnull()``: Opposite of ``isnull()``
- ``dropna()``: Return a filtered version of the data
- ``fillna()``: Return a copy of the data with missing values filled or imputed

We will conclude this section with a brief exploration and demonstration of these routines.


Operando con valores nulos
Como hemos visto, Pandas trata None y NaN como esencialmente intercambiables para indicar valores faltantes o nulos. Para facilitar esta convención, existen varios métodos útiles para detectar, eliminar y reemplazar valores nulos en las estructuras de datos de Pandas. Son:

isnull (): genera una máscara booleana que indica valores faltantes
notnull (): opuesto a isnull ()
dropna (): devuelve una versión filtrada de los datos
fillna (): devuelve una copia de los datos con valores faltantes rellenados o imputados
Concluiremos esta sección con una breve exploración y demostración de estas rutinas.

### Detecting null values
Pandas data structures have two useful methods for detecting null data: ``isnull()`` and ``notnull()``.
Either one will return a Boolean mask over the data. For example:


Detectar valores nulos
Las estructuras de datos de Pandas tienen dos métodos útiles para detectar datos nulos: isnull () y notnull (). Cualquiera de los dos devolverá una máscara booleana sobre los datos. Por ejemplo:

In [13]:
data = pd.Series([1, np.nan, 'hello', None])

In [14]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

As mentioned in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb), Boolean masks can be used directly as a ``Series`` or ``DataFrame`` index:

Como se menciona en Indización y selección de datos, las máscaras booleanas se pueden usar directamente como un índice de serie o de marco de datos:

In [15]:
data[data.notnull()]

0        1
2    hello
dtype: object

The ``isnull()`` and ``notnull()`` methods produce similar Boolean results for ``DataFrame``s.


Los métodos isnull () y notnull () producen resultados booleanos similares para DataFrames

### Dropping null values

In addition to the masking used before, there are the convenience methods, ``dropna()``
(which removes NA values) and ``fillna()`` (which fills in NA values). For a ``Series``,
the result is straightforward:


Descarte de valores nulos
Además del enmascaramiento utilizado anteriormente, existen los métodos de conveniencia, dropna () (que elimina los valores de NA) y fillna () (que completa los valores de NA). Para una serie, el resultado es sencillo:

In [16]:
data.dropna()

0        1
2    hello
dtype: object

For a ``DataFrame``, there are more options.
Consider the following ``DataFrame``:


Para un DataFrame, hay más opciones. Considere el siguiente DataFrame:

In [17]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


We cannot drop single values from a ``DataFrame``; we can only drop full rows or full columns.
Depending on the application, you might want one or the other, so ``dropna()`` gives a number of options for a ``DataFrame``.

By default, ``dropna()`` will drop all rows in which *any* null value is present:


No podemos descartar valores individuales de un DataFrame; solo podemos soltar filas completas o columnas completas. Dependiendo de la aplicación, es posible que desee una u otra, por lo que dropna () ofrece una serie de opciones para un DataFrame.

De forma predeterminada, dropna () eliminará todas las filas en las que esté presente cualquier valor nulo:

In [18]:
df.dropna()

,0,1,2
1,2.0,3.0,5


Alternatively, you can drop NA values along a different axis; ``axis=1`` drops all columns containing a null value:


Alternativamente, puede soltar los valores de NA a lo largo de un eje diferente; axis = 1 elimina todas las columnas que contienen un valor nulo:

In [19]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


But this drops some good data as well; you might rather be interested in dropping rows or columns with *all* NA values, or a majority of NA values.
This can be specified through the ``how`` or ``thresh`` parameters, which allow fine control of the number of nulls to allow through.

The default is ``how='any'``, such that any row or column (depending on the ``axis`` keyword) containing a null value will be dropped.
You can also specify ``how='all'``, which will only drop rows/columns that are *all* null values:


Pero esto también arroja algunos buenos datos; es posible que le interese soltar filas o columnas con todos los valores de NA o la mayoría de los valores de NA. Esto se puede especificar a través de los parámetros how o thresh, que permiten un control preciso de la cantidad de nulos para permitir.

El valor predeterminado es how = 'any', de modo que cualquier fila o columna (dependiendo de la palabra clave del eje) que contenga un valor nulo se descartará. También puede especificar how = 'all', que solo eliminará filas / columnas que son todos valores nulos:

In [20]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [21]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


For finer-grained control, the ``thresh`` parameter lets you specify a minimum number of non-null values for the row/column to be kept:


Para un control más detallado, el parámetro thresh le permite especificar un número mínimo de valores no nulos para la fila / columna que se mantendrá:

In [22]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


Here the first and last row have been dropped, because they contain only two non-null values.


Aquí se han descartado la primera y la última fila, porque contienen solo dos valores no nulos.

### Filling null values

Sometimes rather than dropping NA values, you'd rather replace them with a valid value.
This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values.
You could do this in-place using the ``isnull()`` method as a mask, but because it is such a common operation Pandas provides the ``fillna()`` method, which returns a copy of the array with the null values replaced.

Consider the following ``Series``:


Relleno de valores nulos
A veces, en lugar de descartar los valores de NA, preferiría reemplazarlos con un valor válido. Este valor puede ser un número único como cero, o puede ser algún tipo de imputación o interpolación de los buenos valores. Puede hacerlo en el lugar utilizando el método isnull () como máscara, pero debido a que es una operación tan común, Pandas proporciona el método fillna (), que devuelve una copia de la matriz con los valores nulos reemplazados

In [23]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

We can fill NA entries with a single value, such as zero:

Podemos llenar entradas de NA con un solo valor, como cero:

In [24]:
data.fillna(0)  #relleno las filas b y d que eran nulos con 0

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

We can specify a forward-fill to propagate the previous value forward:
Podemos especificar un relleno hacia adelante para propagar el valor anterior hacia adelante:

In [25]:
# forward-fill
data.fillna(method='ffill') #relleno las filas b y d que eran nulos con el valor de a y de c

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or we can specify a back-fill to propagate the next values backward:

O podemos especificar un relleno para propagar los siguientes valores hacia atrás:

In [26]:
# back-fill
data.fillna(method='bfill') #relleno las filas b y d que eran nulos con el valor de c y de e

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

For ``DataFrame``s, the options are similar, but we can also specify an ``axis`` along which the fills take place:


Para DataFrames, las opciones son similares, pero también podemos especificar un eje a lo largo del cual tienen lugar los rellenos:

In [27]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [28]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


Notice that if a previous value is not available during a forward fill, the NA value remains.

Tenga en cuenta que si un valor anterior no está disponible durante un relleno directo, el valor de NA permanece.

<!--NAVIGATION-->
< [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) | [Contents](Index.ipynb) | [Hierarchical Indexing](03.05-Hierarchical-Indexing.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.04-Missing-Values.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
